# Capítulo 4

In [1]:
spark

Intitializing Scala interpreter ...

Spark Web UI available at http://5fcc63a3e7cf:4040
SparkContext available as 'sc' (version = 3.2.1, master = local[*], app id = local-1655824783105)
SparkSession available as 'spark'


res0: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@637b15e9


In [3]:
sc

res2: org.apache.spark.SparkContext = org.apache.spark.SparkContext@577835d0


## Leer DataSet en una Vista Temporal

In [11]:
import org.apache.spark.sql.SparkSession
val spark = SparkSession
.builder
.appName("SparkSQLExampleApp")
.enableHiveSupport()
.getOrCreate()

import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@637b15e9


In [3]:
val df = spark.read.format("csv")
    .option("inferSchema", "true")
    .option("header", "true")
    .load("departuredelays.csv")

df: org.apache.spark.sql.DataFrame = [date: int, delay: int ... 3 more fields]


In [4]:
// Creacion de la vista temporal
df.createOrReplaceTempView("tabla")

In [7]:
// También se podría haber inferido un squema, ddl
// val schema = "date STRING, delay INT, distance INT, origin STRING, destination STRING"

In [8]:
df.printSchema()

root
 |-- date: integer (nullable = true)
 |-- delay: integer (nullable = true)
 |-- distance: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- destination: string (nullable = true)



In [9]:
df.show(2,false)

+-------+-----+--------+------+-----------+
|date   |delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1011245|6    |602     |ABE   |ATL        |
|1020600|-8   |369     |ABE   |DTW        |
+-------+-----+--------+------+-----------+
only showing top 2 rows



The US flight delays data set has five columns:
- The date column contains a string like 02190925. When converted, this maps to 02-19 09:25 am.
- The delay column gives the delay in minutes between the scheduled and actual departure times. Early departures show negative numbers.
- The distance column gives the distance in miles from the origin airport to the destination airport.
- The origin column contains the origin IATA airport code.
- The destination column contains the destination IATA airport code.

## Queries Básicas

In [5]:
// Buscamos los vuelos con distancia > 1000
spark.sql("""select distance, origin, destination
             from tabla
             where distance > 1000
             order by distance desc""").show(5)

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 5 rows



In [6]:
// Buscamos todos los vuelos entre San Francisco y Chicago
spark.sql("""select origin, destination, delay
             from tabla
             where origin = 'SFO' and destination = 'ORD' and delay > 120
             order by delay desc""").show(5)

+------+-----------+-----+
|origin|destination|delay|
+------+-----------+-----+
|   SFO|        ORD| 1638|
|   SFO|        ORD|  396|
|   SFO|        ORD|  326|
|   SFO|        ORD|  320|
|   SFO|        ORD|  297|
+------+-----------+-----+
only showing top 5 rows



In [7]:
spark.sql("""select origin, destination, delay, cast(cast(date as string) as date)
             from tabla
              """).show(5)

+------+-----------+-----+----------------------------------+
|origin|destination|delay|CAST(CAST(date AS STRING) AS DATE)|
+------+-----------+-----+----------------------------------+
|   ABE|        ATL|    6|                    +1011245-01-01|
|   ABE|        DTW|   -8|                    +1020600-01-01|
|   ABE|        ATL|   -2|                    +1021245-01-01|
|   ABE|        ATL|   -4|                    +1020605-01-01|
|   ABE|        ATL|   -4|                    +1031245-01-01|
+------+-----------+-----+----------------------------------+
only showing top 5 rows



## CASE

In [14]:
spark.sql("""select * from tabla""").show(2)

+-------+-----+--------+------+-----------+
|   date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1011245|    6|     602|   ABE|        ATL|
|1020600|   -8|     369|   ABE|        DTW|
+-------+-----+--------+------+-----------+
only showing top 2 rows



spark.sql("""SELECT delay, origin, destination,
CASE
WHEN delay > 360 THEN 'Very Long Delays'
WHEN delay > 120 AND delay < 360 THEN 'Long Delays'
WHEN delay > 60 AND delay < 120 THEN 'Short Delays'
WHEN delay > 0 and delay < 60 THEN 'Tolerable Delays'
WHEN delay = 0 THEN 'No Delays'
ELSE 'Early'
END AS Flight_Delays
FROM tabla ORDER BY origin, delay DESC""").show(10)

In [54]:
spark.sql(""" select delay, origin, destination,
                                                 CASE
                                                     when delay > 360 then 'mucho retraso'
                                                     when delay between 120 and 360 then 'Buen retraso'
                                                     when delay between 60 and 120 then 'Algo de retraso'
                                                     when delay between 0 and 60 then 'miniretraso'
                                                     when delay == 0 then 'a tiempo'
                                                     else 'pronto'

                                                 END as retrasos
             from tabla
             order by origin, delay desc""")show(10)

+-----+------+-----------+------------+
|delay|origin|destination|    retrasos|
+-----+------+-----------+------------+
|  333|   ABE|        ATL|Buen retraso|
|  305|   ABE|        ATL|Buen retraso|
|  275|   ABE|        ATL|Buen retraso|
|  257|   ABE|        ATL|Buen retraso|
|  247|   ABE|        ATL|Buen retraso|
|  247|   ABE|        DTW|Buen retraso|
|  219|   ABE|        ORD|Buen retraso|
|  211|   ABE|        ATL|Buen retraso|
|  197|   ABE|        DTW|Buen retraso|
|  192|   ABE|        ORD|Buen retraso|
+-----+------+-----------+------------+
only showing top 10 rows



## Crear DATABASE y TABLAS (managed & unmanaged)

### Crear database

In [8]:
spark.sql("create database learn_spark")

res5: org.apache.spark.sql.DataFrame = []


In [9]:
spark.sql("use learn_spark")

res6: org.apache.spark.sql.DataFrame = []


### Crear Managed Table

In [12]:
spark.sql("create table tabla(date string, delay int, distance int, origin string, destination string)")

org.apache.spark.sql.AnalysisException:  Hive support is required to CREATE Hive TABLE (AS SELECT);

In [28]:
spark.sql("""CREATE TABLE tablaunmanaged3(date STRING, delay INT,
distance INT, origin STRING, destination STRING)
USING csv OPTIONS (PATH 'departuredelays.csv')""")

res25: org.apache.spark.sql.DataFrame = []


In [29]:
spark.sql("""select * 
from tablaunmanaged3""").show(5)

+----+-----+--------+------+-----------+
|date|delay|distance|origin|destination|
+----+-----+--------+------+-----------+
+----+-----+--------+------+-----------+



## Creacion VISTAS

In [33]:
spark.sql("""create or replace temp view tempTabla as 
            select date, delay, origin, destination
            from tabla
            where origin = 'JFK'""")

res29: org.apache.spark.sql.DataFrame = []


In [34]:
spark.sql("""select *
            from tempTabla""").show(3)

+-------+-----+------+-----------+
|   date|delay|origin|destination|
+-------+-----+------+-----------+
|1010900|   14|   JFK|        LAX|
|1011200|   -3|   JFK|        LAX|
|1011900|    2|   JFK|        LAX|
+-------+-----+------+-----------+
only showing top 3 rows



In [37]:
spark.sql("""drop view if exists tempTabla""")

res33: org.apache.spark.sql.DataFrame = []


## Metadatos

In [42]:
spark.sql("""show tables""").show(false)

+-----------+---------------------+-----------+
|namespace  |tableName            |isTemporary|
+-----------+---------------------+-----------+
|learn_spark|tablaunmanaged       |false      |
|learn_spark|tablaunmanaged2      |false      |
|learn_spark|tablaunmanaged3      |false      |
|learn_spark|unmanagedtablaejemplo|false      |
|           |tabla                |true       |
+-----------+---------------------+-----------+



In [44]:
spark.sql("show databases").show()

+-----------+
|  namespace|
+-----------+
|    default|
|learn_spark|
+-----------+



In [46]:
spark.sql("show views").show()

+---------+--------+-----------+
|namespace|viewName|isTemporary|
+---------+--------+-----------+
|         |   tabla|       true|
+---------+--------+-----------+



In [48]:
spark.catalog.listDatabases().show()

+-----------+----------------+--------------------+
|       name|     description|         locationUri|
+-----------+----------------+--------------------+
|    default|default database|file:/home/jovyan...|
|learn_spark|                |file:/home/jovyan...|
+-----------+----------------+--------------------+



In [50]:
spark.catalog.listTables().show()

+--------------------+-----------+-----------+---------+-----------+
|                name|   database|description|tableType|isTemporary|
+--------------------+-----------+-----------+---------+-----------+
|      tablaunmanaged|learn_spark|       null| EXTERNAL|      false|
|     tablaunmanaged2|learn_spark|       null| EXTERNAL|      false|
|     tablaunmanaged3|learn_spark|       null| EXTERNAL|      false|
|unmanagedtablaeje...|learn_spark|       null|  MANAGED|      false|
|               tabla|       null|       null|TEMPORARY|       true|
+--------------------+-----------+-----------+---------+-----------+



In [52]:
spark.catalog.listColumns("tabla").show()

+-----------+-----------+--------+--------+-----------+--------+
|       name|description|dataType|nullable|isPartition|isBucket|
+-----------+-----------+--------+--------+-----------+--------+
|       date|       null|     int|    true|      false|   false|
|      delay|       null|     int|    true|      false|   false|
|   distance|       null|     int|    true|      false|   false|
|     origin|       null|  string|    true|      false|   false|
|destination|       null|  string|    true|      false|   false|
+-----------+-----------+--------+--------+-----------+--------+



## Caching SQL tables

In [59]:
spark.sql("cache table tabla")

res54: org.apache.spark.sql.DataFrame = []


In [60]:
spark.sql("uncache table tabla")

res55: org.apache.spark.sql.DataFrame = []


## Pasar de tablas a DataFrames

In [62]:
val tabladf = spark.sql("""select * from tabla""")

tabladf: org.apache.spark.sql.DataFrame = [date: int, delay: int ... 3 more fields]


In [63]:
tabladf.show(2)

+-------+-----+--------+------+-----------+
|   date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1011245|    6|     602|   ABE|        ATL|
|1020600|   -8|     369|   ABE|        DTW|
+-------+-----+--------+------+-----------+
only showing top 2 rows

